# Background replacement using mask prompt

One way to replace the background in an image is to use the "OUTPAINTING" task type with a `maskPrompt` describing the object(s) to keep and a text prompt describing the new background. A common use case for this feature is placing a real product in a generated environment.

The script below will use the image on the left as input. It will replace the background behind the coffee maker by using the mask prompt *"coffee maker"* and the text prompt *"a coffee maker in a sparse stylish kitchen, a single plate of pastries next to the coffee maker, a single cup of coffee. highly detailed, highest quality, product imagery"*. The result will look similar to the image on the right. Note the realistic shadows and reflections that ground the original coffee maker into the generated scene.

<div style="display: flex; justify-content: space-between;">
    <div style="width: 48%;">
        <p align="center">
            <img src="../images/amazon-coffee-maker-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Input image</em>
        </p>
    </div>
    <div style="width: 48%;">
        <p align="center">
            <img src="../images/doc-images/background-replacement-mask-prompt-example-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Output image</em>
        </p>
    </div>
</div>


Edit the `text` prompt to experiment with putting the coffee maker in different settings.

In [ ]:
from random import randint
from amazon_image_gen import BedrockImageGenerator
import file_utils
import logging
import base64
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

# Set the image to be edited.
source_image = "../images/amazon-coffee-maker-1.png"

# Load the input image from disk.
with open(source_image, "rb") as image_file:
    source_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

# Configure the inference parameters.
inference_params = {
    "taskType": "OUTPAINTING",
    "outPaintingParams": {
        "image": source_image_base64,
        "text": "a coffee maker in a sparse stylish kitchen, a single plate of pastries next to the coffee maker, a single cup of coffee. highly detailed, highest quality, product imagery",  # Description of the background to generate
        "maskPrompt": "coffee maker",  # The element(s) to keep
        "outPaintingMode": "PRECISE",  # "DEFAULT" softens the mask. "PRECISE" keeps it sharp.
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,  # Number of variations to generate. 1 to 5.
        "quality": "standard",  # Allowed values are "standard" and "premium"
        "cfgScale": 7.0,  # How closely the prompt will be followed
        "seed": randint(0, 858993459),  # Use a random seed
    },
}

# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(
    output_directory=output_directory
)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(response["images"], output_directory, "image")
    for image in images:
        display(image)